In [3]:
# ✅ 크롤링 코드 수정: \n 기반 문단 구분 포함
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pymysql
import time
import re
from datetime import datetime

# ✅ 셀레니움 드라이버 설정
driver_path = "C:/Users/user/Desktop/RefrigeratorCode/chromedriver-win64/chromedriver.exe"
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# ✅ pymysql로 MySQL 연결 설정
db = pymysql.connect(
    host='localhost',
    user='root',
    password='sk784512!!',
    db='refrigerator',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)
cursor = db.cursor()

insert_query = """
INSERT IGNORE INTO recipes
(title, link, content, author, thumbnail, platform, likes, comments, post_time, collected_at)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


# ✅ 키워드 필터
KEYWORDS = ["레시피", "만드는", "만들기", "요리", "끓이", "하는법"]

def contains_keyword(title):
    return any(kw in title for kw in KEYWORDS)

# ✅ 시간 텍스트 변환 함수
def convert_post_time(time_text):
    now = datetime.now()
    try:
        if "시간 전" in time_text:
            hours = int(re.search(r"(\d+)", time_text).group(1))
            post_date = now - timedelta(hours=hours)
        elif "분 전" in time_text:
            minutes = int(re.search(r"(\d+)", time_text).group(1))
            post_date = now - timedelta(minutes=minutes)
        elif "일 전" in time_text:
            days = int(re.search(r"(\d+)", time_text).group(1))
            post_date = now - timedelta(days=days)
        else:
            post_date = datetime.strptime(time_text.strip(), "%Y. %m. %d.")
        return post_date.strftime("%Y-%m-%d")
    except:
        return None

# ✅ 본문 수집
for page in range(1, 101):
    print(f"{page} / 100 페이지 수집 중... {round(page / 100 * 100)}% 완료")
    url = f"https://section.blog.naver.com/ThemePost.naver?directoryNo=20&activeDirectorySeq=2&currentPage={page}"
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    posts = soup.select("div.info_post")

    for post in posts:
        try:
            title_tag = post.select_one("strong.title_post")
            if not title_tag:
                continue
            title = title_tag.get_text(strip=True)
            if not contains_keyword(title):
                continue

            link_tag = post.select_one("a.desc_inner")
            link = link_tag["href"] if link_tag else ""

            author_tag = post.select_one("em.name_author")
            author = author_tag.get_text(strip=True) if author_tag else "Unknown"

            time_tag = post.select_one("span.time")
            raw_time = time_tag.get_text(strip=True) if time_tag else ""
            post_time = convert_post_time(raw_time)

            like_tag = post.select_one("span.like em")
            likes = int(like_tag.get_text(strip=True).replace(",", "")) if like_tag else 0

            reply_tag = post.select_one("span.reply em")
            comments = int(reply_tag.get_text(strip=True).replace(",", "")) if reply_tag else 0

            driver.get(link)
            time.sleep(3)
            try:
                driver.switch_to.frame("mainFrame")
                time.sleep(1)
            except:
                pass

            detail_soup = BeautifulSoup(driver.page_source, "html.parser")
            content = ""
            viewer = detail_soup.find("div", class_="se-viewer")
            if viewer:
                paragraphs = viewer.find_all("p", class_="se-text-paragraph")
                text_lines = []
                seen_lines = set()
                for p in paragraphs:
                    span = p.find("span")
                    if span:
                        text = span.get_text(strip=True)
                        if text and text not in seen_lines:
                            seen_lines.add(text)
                            text_lines.append(text)
                content = "\n".join(text_lines)  # ✅ 줄바꿈 포함 저장

            thumb_tag = detail_soup.select_one("img.se-image-resource") or \
                        detail_soup.select_one("div#postViewArea img") or \
                        detail_soup.select_one("div#contentArea img")
            thumbnail = thumb_tag["src"] if thumb_tag else ""

            # 수집 시점 정의
            collected_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            cursor.execute(insert_query, (
                title, link, content, author, thumbnail, "naver", likes, comments, post_time, collected_at
            ))

            db.commit()

        except Exception as e:
            print("오류 발생:", e)
            continue

# ✅ 마무리
driver.quit()
cursor.close()
db.close()
print("✅ 크롤링 및 MySQL 저장 완료!")


1 / 100 페이지 수집 중... 1% 완료
2 / 100 페이지 수집 중... 2% 완료
3 / 100 페이지 수집 중... 3% 완료
4 / 100 페이지 수집 중... 4% 완료
5 / 100 페이지 수집 중... 5% 완료
6 / 100 페이지 수집 중... 6% 완료
7 / 100 페이지 수집 중... 7% 완료
8 / 100 페이지 수집 중... 8% 완료
9 / 100 페이지 수집 중... 9% 완료
10 / 100 페이지 수집 중... 10% 완료
11 / 100 페이지 수집 중... 11% 완료
12 / 100 페이지 수집 중... 12% 완료
13 / 100 페이지 수집 중... 13% 완료
14 / 100 페이지 수집 중... 14% 완료
15 / 100 페이지 수집 중... 15% 완료
16 / 100 페이지 수집 중... 16% 완료
17 / 100 페이지 수집 중... 17% 완료
18 / 100 페이지 수집 중... 18% 완료
19 / 100 페이지 수집 중... 19% 완료
20 / 100 페이지 수집 중... 20% 완료
21 / 100 페이지 수집 중... 21% 완료
22 / 100 페이지 수집 중... 22% 완료
23 / 100 페이지 수집 중... 23% 완료
24 / 100 페이지 수집 중... 24% 완료
25 / 100 페이지 수집 중... 25% 완료
26 / 100 페이지 수집 중... 26% 완료
27 / 100 페이지 수집 중... 27% 완료
28 / 100 페이지 수집 중... 28% 완료
29 / 100 페이지 수집 중... 29% 완료
30 / 100 페이지 수집 중... 30% 완료
31 / 100 페이지 수집 중... 31% 완료
32 / 100 페이지 수집 중... 32% 완료
33 / 100 페이지 수집 중... 33% 완료
34 / 100 페이지 수집 중... 34% 완료
35 / 100 페이지 수집 중... 35% 완료
36 / 100 페이지 수집 중... 36% 완료
37 / 100 페